# このページについて
[The Expressive Power of Low-Rank Adaptation](https://arxiv.org/abs/2310.17513)についてまとめます．
# イントロ
Low rank adaptationとは，LLMのフルファインチューニングに対して，効率の良いファインチューニングのアプローチです．事前学習された重み行列に軽量な低ランクなアダプタを作用させます．性能はかなり良く，LoRAはフルファインチューニングに匹敵するかそれより良くなる場合があるらしいです(Hu et al)．LoRAの理論において，以下のような疑問が存在します．
* 事前学習済みモデル(事前モデル)$f$を，ターゲットモデル$\bar{f}$に一致させるために必要なアダプタの最小ランク
* モデルアーキテクチャ(深さや幅)が最小ランクにどれくらい影響するか？
* アダプタランクが最小ランクより小さい場合に発生する近似誤差
この研究では，以下のような定理が成り立つことを主張しています．
**定理1**
$\bar{f}$をターゲットFNN(全結合ニューラルネットワーク)とし，$f_{0}$を任意の凍結FNNとする．ランクとネットワークアーキテクチャに関する条件の下で，低ランクなアダプタが存在し，$f_{0}$を低ランクなアダプタで適用した結果と$\bar{f}$と正確に等しくなる．

**定理2**
$\bar{f}$をターゲットTFNとし，$f_{0}$を凍結TFN(Transformerネットワーク)とする．ランクとネットワークアーキテクチャに関する条件の下で，アテンション重み行列に対する低ランクなアダプタが存在し，$f_{0}$を低ランクなアダプタで適用した結果と$\bar{f}$と正確に等しくなる．

# 準備
## 記法

| 記号/概念                                              | 定義/説明                                                                                                                                                                                                  | 補足                                                                                                                                                                          |
| :------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :-------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| $[N]$                                              | $\{1,2, \ldots, N\}$                                                                                                                                                                                   | 自然数の集合                                                                                                                                                                      |
| $\wedge$                                           | 最小関数                                                                                                                                                                                                   | 2つの入力の小さい方を取る．例: $a \wedge b = \min(a,b)$                                                                                                                                   |
| $\vee$                                             | 最大関数                                                                                                                                                                                                   | 2つの入力の大きい方を取る．例: $a \vee b = \max(a,b)$                                                                                                                                     |
| $\boldsymbol{I}$                                   | 単位行列                                                                                                                                                                                                   | 文脈からサイズが明らかな場合に使用．                                                                                                                                                          |
| $\prod_{l=1}^{L} \boldsymbol{W}_{l}$               | 行列の積 $\boldsymbol{W}_{L} \boldsymbol{W}_{L-1} \cdots \boldsymbol{W}_{1}$                                                                                                                               | 行列は添え字の大きい方 (左側) から小さい方 (右側) へ乗算される．                                                                                                                                        |
| $\sum_{i=m}^{n} a_{i}$ ($m>n$, スカラー)               | $0$ (空和)                                                                                                                                                                                               |                                                                                                                                                                             |
| $\prod_{i=m}^{n} a_{i}$ ($m>n$, スカラー)              | $1$ (空積)                                                                                                                                                                                               |                                                                                                                                                                             |
| $\sum_{i=m}^{n} \boldsymbol{W}_{i}$ ($m>n$, 正方行列)  | $\boldsymbol{O}$ (ゼロ行列)                                                                                                                                                                                |                                                                                                                                                                             |
| $\prod_{i=m}^{n} \boldsymbol{W}_{i}$ ($m>n$, 正方行列) | $\boldsymbol{I}$ (単位行列)                                                                                                                                                                                |                                                                                                                                                                             |
| $\boldsymbol{W}=\boldsymbol{U D V}^{\top}$         | 行列 $\boldsymbol{W}$ の特異値分解 (SVD)                                                                                                                                                                       | $\boldsymbol{U}, \boldsymbol{V} \in \mathbb{R}^{D \times D}$ は正規直交行列，$\boldsymbol{D} \in \mathbb{R}^{D \times D}$ は対角行列 (ここでは $\boldsymbol{W}$ が $D \times D$ の正方行列の場合を想定)． |
| $\sigma_{d}(\boldsymbol{W})$                       | $\boldsymbol{W}$ の $d$ 番目に大きな特異値． $\sigma_{1}(\boldsymbol{W}) \geq \sigma_{2}(\boldsymbol{W}) \geq \cdots \geq \sigma_{D}(\boldsymbol{W}) \geq 0$．                                                     | $d \in [D]$．$d>D$ の場合，$\sigma_{d}(\boldsymbol{W})=0$ と定義される．                                                                                                                |
| $\mathrm{LR}_{r}(\boldsymbol{W})$                  | 行列 $\boldsymbol{W}$ の最良のランク $r$ 近似．$\sum_{i=1}^{r} \sigma_{i} \boldsymbol{u}_{i} \boldsymbol{v}_{i}^{T}$ ($\boldsymbol{u}_{i}, \boldsymbol{v}_{i}$ はそれぞれ $\boldsymbol{U}, \boldsymbol{V}$ の $i$ 番目の列)． | $r \geq \operatorname{rank}(\boldsymbol{W})$ の場合，$\mathrm{LR}_{r}(\boldsymbol{W})=\boldsymbol{W}$．添え字 $r$ が省略されることもある．                                                      |
**特異値分解復習**
特異値分解$\boldsymbol{A} = \boldsymbol{U} \boldsymbol{\Sigma} \boldsymbol{V}^{\top}$とは，行列の分解法です．**どのような行列もある3つの行列の積に分解**できます．
* $\boldsymbol{U}$ ：左特異行列
	* $m$行$m$列の直交行列です．
* $\boldsymbol{\Sigma}$：特異値行列
	* $m$行$n$列の対角行列です．対角成分には特異値という非負の実数が大きい順に並んでいます．
* $\boldsymbol{V}^{\top}$ ：右特異行列
	* $n$行$n$列の直交行列です．

# LoRAを用いた線形モデルの表現力(LoRAでどれだけターゲットモデルを表現できるか)
## 線形なモデル
ターゲットモデルと，凍結モデルがどちらも線形である場合を考えます．つまり，
$$
\quad \bar{f}(\boldsymbol{x})=\overline{\boldsymbol{W}} \boldsymbol{x}, \quad  \quad f_{0}(\boldsymbol{x})=\boldsymbol{W}_{L} \cdots \boldsymbol{W}_{1} \boldsymbol{x}=\left(\prod_{l=1}^{L} \boldsymbol{W}_{l}\right) \boldsymbol{x} .
$$
ここで，
* $\overline{\boldsymbol{W}}, \boldsymbol{W}_{1}, \ldots, \boldsymbol{W}_{L} \in \mathbb{R}^{D \times D}$
* $\bar{f}$：単一層
* $f_{0}$：$L$層
* 両モデルのバイアスベクトル：$0$
* 活性化関数：線形
* 両モデルの各層：同じ数のニューロンを持つ
## LoRAの適用
ランク$R \in[D]$のLoRAアダプタを以下のように定義します．(記法がよろしくない．)
$$
\Delta \boldsymbol{W}_{1}, \ldots, \Delta \boldsymbol{W}_{L}
$$
任意の$l \in[L]$に対して$\operatorname{rank}\left(\Delta \boldsymbol{W}_{l}\right) \leq R$です．LoRAアダプタを凍結モデルに適用します．適用後のモデルを適応モデル$f$と言い，以下のように定義します．
$$
\quad f(\boldsymbol{x})=\left(\boldsymbol{W}_{L}+\Delta \boldsymbol{W}_{L}\right) \cdots\left(\boldsymbol{W}_{1}+\Delta \boldsymbol{W}_{1}\right) \boldsymbol{x},
$$
各層でアダプタを凍結モデルに線形結合していますね．

## LoRA適応モデルとターゲットモデルの誤差
LoRAの理論解析をするために，以下の誤差を最小化するような最適化問題を考えます．
$$
\prod_{l=1}^{L}\left(\boldsymbol{W}_{l}+\Delta \boldsymbol{W}_{l}\right)-\overline{\boldsymbol{W}}
$$
ここで，以下のような補題が成り立つことを確認しましょう．
## 補題1
誤差行列$\boldsymbol{E}:=\overline{\boldsymbol{W}}-\prod_{l=1}^{L} \boldsymbol{W}_{l}$を定義し，ランクを$R_{\boldsymbol{E}}=\operatorname{rank}(\boldsymbol{E})$とします．LoRAランク$R \in[D]$に対して，凍結モデルのすべての**重み**行列$\left(\boldsymbol{W}_{l}\right)_{l=1}^{L}$および$\prod_{l=1}^{L} \boldsymbol{W}_{l}+\mathrm{LR}_{r}(\boldsymbol{E})$がすべての$r \leq R(L-1)$に対して正則であると仮定すると，以下が成り立ちます．

$$
\min _{\Delta \boldsymbol{W}_{l}: \operatorname{rank}\left(\Delta \boldsymbol{W}_{l}\right) \leq R}\left\|\prod_{l=1}^{L}\left(\boldsymbol{W}_{l}+\Delta \boldsymbol{W}_{l}\right)-\overline{\boldsymbol{W}}\right\|_{2}=\sigma_{R L+1}(\boldsymbol{E}) .
$$
LoRAアダプタの各層のランクを $R$ 以下に制限した場合，LoRAで適応させたモデルとターゲットモデルの差の最小値は，$\boldsymbol{E}$ の $(RL+1)$ 番目の特異値と等しくなるという意味です．

## 補題1の証明

以下の最適化問題を解くことを考えます．

$$
\min _{\Delta \boldsymbol{W}_{l}: \operatorname{rank}\left(\Delta \boldsymbol{W}_{l}\right) \leq R}\left\|\prod_{l=1}^{L}\left(\boldsymbol{W}_{l}+\Delta \boldsymbol{W}_{l}\right)-\overline{\boldsymbol{W}}\right\|_{2}
$$

この式の両方の項から，$\prod_{l=1}^{L} \boldsymbol{W}_{l}$を引きます．

$$
\min _{\Delta \boldsymbol{W}_{l}: \operatorname{rank}\left(\Delta \boldsymbol{W}_{l}\right) \leq R}\|\underbrace{\left(\prod_{l=1}^{L}\left(\boldsymbol{W}_{l}+\Delta \boldsymbol{W}_{l}\right)-\prod_{l=1}^{L} \boldsymbol{W}_{l}\right)}_{:=\boldsymbol{A}}-\underbrace{\left(\overline{\boldsymbol{W}}-\prod_{l=1}^{L} \boldsymbol{W}_{l}\right)}_{:=\boldsymbol{E}}\|_{2} \quad \cdots(2)
$$

ここで，$\boldsymbol{A}$はLoRAアダプタによって加えられた変化分全体を表し，$\boldsymbol{E}$は目標モデルと元の凍結モデルとの間の誤差行列を表します．

式(2)を分析するために，まず$\boldsymbol{A}$の構造を詳しく見ていきましょう．$\boldsymbol{A}$は次のように展開できます．

$$
\begin{aligned}
\boldsymbol{A} &= \prod_{k=1}^{L}(\boldsymbol{W}_{k}+\Delta \boldsymbol{W}_{k}) - \prod_{k=1}^{L} \boldsymbol{W}_{k}\\
&= (\boldsymbol{W}_{L}+\Delta \boldsymbol{W}_{L})(\boldsymbol{W}_{L-1}+\Delta \boldsymbol{W}_{L-1})\cdots(\boldsymbol{W}_{1}+\Delta \boldsymbol{W}_{1}) - \boldsymbol{W}_{L}\boldsymbol{W}_{L-1}\cdots\boldsymbol{W}_{1}
\end{aligned}
$$
一番左の項 $(\boldsymbol{W}_{L}+\Delta \boldsymbol{W}_{L})$ を展開します．$(A + B )(CDE) =ACDE+BCDE$　なので，
$$
\boldsymbol{A} = \boldsymbol{W}_{L}(\boldsymbol{W}_{L-1}+\Delta \boldsymbol{W}_{L-1})\cdots(\boldsymbol{W}_{1}+\Delta \boldsymbol{W}_{1}) + \Delta \boldsymbol{W}_{L}(\boldsymbol{W}_{L-1}+\Delta \boldsymbol{W}_{L-1})\cdots(\boldsymbol{W}_{1}+\Delta \boldsymbol{W}_{1}) - \prod_{k=1}^{L} \boldsymbol{W}_{k}
$$

ここで， $\boldsymbol{W}_{L}(\boldsymbol{W}_{L-1}+\Delta \boldsymbol{W}_{L-1})\cdots(\boldsymbol{W}_{1}+\Delta \boldsymbol{W}_{1})$ と，最後の項 $-\prod_{k=1}^{L} \boldsymbol{W}_{k}$ に注目します．$\prod_{k=1}^{L-1}(\boldsymbol{W}_{k}+\Delta \boldsymbol{W}_{k}) = (\boldsymbol{W}_{L-1}+\Delta \boldsymbol{W}_{L-1})\cdots(\boldsymbol{W}_{1}+\Delta \boldsymbol{W}_{1})$ と書くことにすると，
$$
\boldsymbol{A} = \boldsymbol{W}_{L} \left( \prod_{k=1}^{L-1}(\boldsymbol{W}_{k}+\Delta \boldsymbol{W}_{k}) \right) + \Delta \boldsymbol{W}_{L} \left( \prod_{k=1}^{L-1}(\boldsymbol{W}_{k}+\Delta \boldsymbol{W}_{k}) \right) - \boldsymbol{W}_{L} \left( \prod_{k=1}^{L-1} \boldsymbol{W}_{k} \right)
$$
となります．最初の項と最後の項から $\boldsymbol{W}_L$ をくくり出すと，
$$
\boldsymbol{A} = \boldsymbol{W}_{L} \left( \prod_{k=1}^{L-1}(\boldsymbol{W}_{k}+\Delta \boldsymbol{W}_{k}) - \prod_{k=1}^{L-1} \boldsymbol{W}_{k} \right) + \Delta \boldsymbol{W}_{L} \left( \prod_{k=1}^{L-1}(\boldsymbol{W}_{k}+\Delta \boldsymbol{W}_{k}) \right)
$$
よって，
$$
\boldsymbol{A} = \Delta \boldsymbol{W}_{L} \prod_{l=1}^{L-1}\left(\boldsymbol{W}_{l}+\Delta \boldsymbol{W}_{l}\right)+\boldsymbol{W}_{L} \left( \prod_{l=1}^{L-1}\left(\boldsymbol{W}_{l}+\Delta \boldsymbol{W}_{l}\right) - \prod_{l=1}^{L-1} \boldsymbol{W}_{l} \right)
$$
更にもう一段階展開すると，
$$
\boldsymbol{A} = \Delta \boldsymbol{W}_{L} \prod_{l=1}^{L-1}\left(\boldsymbol{W}_{l}+\Delta \boldsymbol{W}_{l}\right) + \boldsymbol{W}_{L} \Delta \boldsymbol{W}_{L-1} \prod_{l=1}^{L-2}\left(\boldsymbol{W}_{l}+\Delta \boldsymbol{W}_{l}\right) + \boldsymbol{W}_{L} \boldsymbol{W}_{L-1} \left( \prod_{l=1}^{L-2}\left(\boldsymbol{W}_{l}+\Delta \boldsymbol{W}_{l}\right) - \prod_{l=1}^{L-2} \boldsymbol{W}_{l} \right)
$$
となります．このパターンを見ると，
*   第1項: $\Delta \boldsymbol{W}_L$ に $(\boldsymbol{W}_{L-1}+\Delta \boldsymbol{W}_{L-1})$ から $(\boldsymbol{W}_1+\Delta \boldsymbol{W}_1)$ までの積がかかっています．
*   第2項: $\boldsymbol{W}_L \Delta \boldsymbol{W}_{L-1}$ に $(\boldsymbol{W}_{L-2}+\Delta \boldsymbol{W}_{L-2})$ から $(\boldsymbol{W}_1+\Delta \boldsymbol{W}_1)$ までの積がかかっています．
*  結果として，$\Delta \boldsymbol{W}_L$の左側の係数に$\boldsymbol{W}_L$が複数回かかり，右辺の係数に$(\boldsymbol{W}_{L-1}+\Delta \boldsymbol{W}_{L-1})$がかかっています．
この展開を再帰的に適用していくと，$\boldsymbol{A}$は最終的に以下のような$L$個の項の和として表すことができます．

$$
\boldsymbol{A} = \sum_{l=1}^{L} \underbrace{\left[\left(\prod_{i=l+1}^{L} \boldsymbol{W}_{i}\right) \Delta \boldsymbol{W}_{l}\left(\prod_{j=1}^{l-1}\left(\boldsymbol{W}_{j}+\Delta \boldsymbol{W}_{j}\right)\right)\right]}_{:=\boldsymbol{A}_{l}} \quad \cdots(3)
$$

ただし，この式(3)において，積の範囲が空になる場合は単位行列 $\boldsymbol{I}$ とします（例：$l=L$のとき $\prod_{i=L+1}^{L} \boldsymbol{W}_{i} = \boldsymbol{I}$，$l=1$のとき $\prod_{j=1}^{0}(\boldsymbol{W}_{j}+\Delta \boldsymbol{W}_{j}) = \boldsymbol{I}$）．

各項 $\boldsymbol{A}_l$ のランクは，$\Delta \boldsymbol{W}_l$ のランクが $R$ 以下であることから，$\operatorname{rank}(\boldsymbol{A}_l) \leq \operatorname{rank}(\Delta \boldsymbol{W}_l) \leq R$ となります．したがって，$\boldsymbol{A}$ はランク $R$ 以下の行列 $L$ 個の和なので，そのランクは $\operatorname{rank}(\boldsymbol{A}) \leq \sum_{l=1}^{L} \operatorname{rank}(\boldsymbol{A}_l) \leq LR$ となります．

Eckart-Young-Mirskyの定理によれば，ある行列 $\boldsymbol{E}$ をランク $k$ 以下の行列で最もよく近似するときの誤差は，$\boldsymbol{E}$ の $(k+1)$ 番目の特異値 $\sigma_{k+1}(\boldsymbol{E})$ で与えられ，その最適な近似行列は $\mathrm{LR}_k(\boldsymbol{E})$ となります．

この定理を考慮すると，式(2)の最適化問題は，$\boldsymbol{A}$ のランクが $LR$ 以下であるという制約の下での近似問題と見なせるので，次のような下界が得られます．

$$
\min_{\Delta \boldsymbol{W}_{l}: \operatorname{rank}(\Delta \boldsymbol{W}_{l}) \leq R} \|\boldsymbol{A}-\boldsymbol{E}\|_{2} \geq \min_{\boldsymbol{A}': \operatorname{rank}(\boldsymbol{A}') \leq LR} \|\boldsymbol{A}'-\boldsymbol{E}\|_{2} = \sigma_{LR+1}(\boldsymbol{E}) \quad \cdots(4)
$$

$\operatorname{rank}(\Delta \boldsymbol{W}_l) \leq R$ という条件を満たしつつ，それによって構成される$\boldsymbol{A}$が，$\boldsymbol{E}$の最良のランク $(LR \wedge R_{\boldsymbol{E}})$ 近似，すなわち $\boldsymbol{A} = \mathrm{LR}_{LR \wedge R_{\boldsymbol{E}}}(\boldsymbol{E})$ となるような $\Delta \boldsymbol{W}_l$ が存在することを示します．ここで，$R_{\boldsymbol{E}} = \operatorname{rank}(\boldsymbol{E})$ です．

$\boldsymbol{E}' = \mathrm{LR}_{LR \wedge R_{\boldsymbol{E}}}(\boldsymbol{E})$ とおきます．以下の等式が成り立つように $\Delta \boldsymbol{W}_l$ を構成することを目指します．

$$
\prod_{l=1}^{L}\left(\boldsymbol{W}_{l}+\Delta \boldsymbol{W}_{l}\right)-\prod_{l=1}^{L} \boldsymbol{W}_{l} = \boldsymbol{E}' \quad \cdots(5)
$$

この式(5)が成り立てば補題が達成されることになります．

$\boldsymbol{E}$ の特異値分解を $\boldsymbol{E} = \boldsymbol{U D V}^{\top}$ とします．すると，$\boldsymbol{E}'$ は $\boldsymbol{E}' = \boldsymbol{U D}' \boldsymbol{V}^{\top}$ と書けます．ここで $\boldsymbol{D}'$ は，$\boldsymbol{D}$ の最初の $(LR \wedge R_{\boldsymbol{E}})$ 個の特異値をそのまま持ち，残りの特異値を0とした行列です．

式(3)の $\boldsymbol{A} = \sum_{l=1}^{L} \boldsymbol{A}_l$ と，$\boldsymbol{E}' = \sum_{l=1}^{L} \boldsymbol{E}'_l$ （ただし $\boldsymbol{E}'_l = \boldsymbol{E}' \boldsymbol{Q}_l$）という分解を考え，各 $l$ について $\boldsymbol{A}_l = \boldsymbol{E}'_l$ となるように $\Delta \boldsymbol{W}_l$ を構成します．
$\boldsymbol{Q}_l$ は，$\boldsymbol{E}'$ の特異値空間を適切に分割するための行列で，次のように定義します．

$$
\boldsymbol{Q}_{l}=\boldsymbol{V} \boldsymbol{I}_{(R(l-1)+1) \wedge R_{\boldsymbol{E}^{\prime}}: Rl \wedge R_{\boldsymbol{E}^{\prime}}, D} \boldsymbol{V}^{\top} \quad \cdots(11)
$$

ここで，$\boldsymbol{I}_{a:b,D}$ は $a$ 番目から $b$ 番目までの対角要素が1で，他が0である $D \times D$ の対角行列です．この $\boldsymbol{Q}_l$ の定義により，$\sum_{l=1}^L \boldsymbol{Q}_l$ は $\boldsymbol{E}'$ の値域（ランク $R_{\boldsymbol{E}'}$ まで）で単位行列のように振る舞い，また $\operatorname{rank}(\boldsymbol{E}'\boldsymbol{Q}_l) \leq R$ が保証されます（$Rl \leq R_{\boldsymbol{E}'}$ の場合）．

次に，$\Delta \boldsymbol{W}_l$ を帰納的に構成します．

**$l=1$ の場合:**
式(6)より，$\boldsymbol{A}_1 = (\prod_{i=2}^{L} \boldsymbol{W}_i) \Delta \boldsymbol{W}_1 (\boldsymbol{I}) = \boldsymbol{E}'_1$ となります．
仮定より，個々の $\boldsymbol{W}_i$ は正則なので，その積である $\prod_{i=2}^{L} \boldsymbol{W}_i$ も正則です．したがって，

$$
\Delta \boldsymbol{W}_1 = \left(\prod_{i=2}^{L} \boldsymbol{W}_i\right)^{-1} \boldsymbol{E}'_1 \quad \cdots(12')
$$

と $\Delta \boldsymbol{W}_1$ を構成できます．このとき，$\operatorname{rank}(\Delta \boldsymbol{W}_1) = \operatorname{rank}(\boldsymbol{E}'_1) \leq R$ となります．
そして，$\boldsymbol{W}_1 + \Delta \boldsymbol{W}_1$ の正則性を確認します．これは，論文の証明で行われているように，$\boldsymbol{W}_1 + \Delta \boldsymbol{W}_1$ を変形すると $\prod_{i=1}^{L} \boldsymbol{W}_i + \boldsymbol{E}'_1$ に関連する行列となり，これは仮定の「$\prod_{l=1}^{L} \boldsymbol{W}_{l}+\mathrm{LR}_{r}(\boldsymbol{E})$ が正則」から，$\mathrm{LR}_{R \wedge R_{\boldsymbol{E}'}}(\boldsymbol{E})$ （$\boldsymbol{E}'_1$ はこれの部分）を考えれば，正則であることが示されます．

**$l>1$ の場合:**
帰納法の仮定として，$k < l$ なる全ての $k$ について，$\Delta \boldsymbol{W}_k$ が構成され，$\prod_{j=1}^{k}(\boldsymbol{W}_j+\Delta \boldsymbol{W}_j)$ が正則であるとします．
式(6)より，$\boldsymbol{A}_l = (\prod_{i=l+1}^{L} \boldsymbol{W}_i) \Delta \boldsymbol{W}_l (\prod_{j=1}^{l-1}(\boldsymbol{W}_j+\Delta \boldsymbol{W}_j)) = \boldsymbol{E}'_l$ です．
$\prod_{i=l+1}^{L} \boldsymbol{W}_i$ は正則であり，帰納法の仮定から $\prod_{j=1}^{l-1}(\boldsymbol{W}_j+\Delta \boldsymbol{W}_j)$ も正則です．したがって，

$$
\Delta \boldsymbol{W}_l = \left(\prod_{i=l+1}^{L} \boldsymbol{W}_i\right)^{-1} \boldsymbol{E}'_l \left(\prod_{j=1}^{l-1}\left(\boldsymbol{W}_j+\Delta \boldsymbol{W}_j\right)\right)^{-1}
$$

と $\Delta \boldsymbol{W}_l$ を構成できます．このとき，$\operatorname{rank}(\Delta \boldsymbol{W}_l) = \operatorname{rank}(\boldsymbol{E}'_l) \leq R$ となります．
次に，$\boldsymbol{W}_l + \Delta \boldsymbol{W}_l$ の正則性も同様に，仮定「$\prod_{l=1}^{L} \boldsymbol{W}_{l}+\mathrm{LR}_{r}(\boldsymbol{E})$ が正則」を用いて示されます．具体的には，$\boldsymbol{W}_l + \Delta \boldsymbol{W}_l$ を含む積 $\prod_{j=1}^{l}(\boldsymbol{W}_j+\Delta \boldsymbol{W}_j)$ の正則性が，$\prod_{i=1}^{L} \boldsymbol{W}_i + \sum_{k=1}^{l} \boldsymbol{E}'_k$ に関連する行列の正則性に帰着され，$\sum_{k=1}^{l} \boldsymbol{E}'_k = \mathrm{LR}_{Rl \wedge R_{\boldsymbol{E}'}}(\boldsymbol{E})$ となることから，仮定が適用できます（$Rl \leq R(L-1)$ を満たす範囲で）．

この帰納的な構成により，全ての $l \in [L]$ について $\boldsymbol{A}_l = \boldsymbol{E}'_l$ を満たす $\Delta \boldsymbol{W}_l$ (ランク $R$ 以下) が存在し，かつ $\boldsymbol{W}_l + \Delta \boldsymbol{W}_l$ (およびその積) が必要な範囲で正則であることが保証されます．
結果として，$\boldsymbol{A} = \sum_{l=1}^L \boldsymbol{A}_l = \sum_{l=1}^L \boldsymbol{E}'_l = \boldsymbol{E}' = \mathrm{LR}_{LR \wedge R_{\boldsymbol{E}}}(\boldsymbol{E})$ が達成されます．

したがって，最小の近似誤差は，
$\|\boldsymbol{A} - \boldsymbol{E}\|_{2} = \|\mathrm{LR}_{LR \wedge R_{\boldsymbol{E}}}(\boldsymbol{E}) - \boldsymbol{E}\|_{2} = \sigma_{(LR \wedge R_{\boldsymbol{E}})+1}(\boldsymbol{E})$
となります．
もし $LR < R_{\boldsymbol{E}}$ ならば，これは $\sigma_{LR+1}(\boldsymbol{E})$ となります．
もし $LR \geq R_{\boldsymbol{E}}$ ならば，$LR \wedge R_{\boldsymbol{E}} = R_{\boldsymbol{E}}$ となり，$\mathrm{LR}_{R_{\boldsymbol{E}}}(\boldsymbol{E}) = \boldsymbol{E}$ ですから，誤差は0です．このとき，$\sigma_{R_{\boldsymbol{E}}+1}(\boldsymbol{E})=0$ であり，$(LR+1)$ 番目以降の特異値も0なので，式は整合します．

もし $R \geq \lceil R_{\boldsymbol{E}}/L \rceil$ であれば，$LR \geq R_{\boldsymbol{E}}$ が成り立ちます．
このとき，$LR \wedge R_{\boldsymbol{E}} = R_{\boldsymbol{E}}$ となるので，上記で構成したように $\boldsymbol{A} = \mathrm{LR}_{R_{\boldsymbol{E}}}(\boldsymbol{E}) = \boldsymbol{E}$ とすることができます．
これは，$\prod_{l=1}^{L}(\boldsymbol{W}_l+\Delta \boldsymbol{W}_l) - \prod_{l=1}^{L} \boldsymbol{W}_l = \boldsymbol{E} = \overline{\boldsymbol{W}} - \prod_{l=1}^{L} \boldsymbol{W}_l$ を意味します．
したがって，$\prod_{l=1}^{L}(\boldsymbol{W}_l+\Delta \boldsymbol{W}_l) = \overline{\boldsymbol{W}}$ となり，$f \equiv \bar{f}$ が達成されます．

QED

$L=1$かつ$R=D$を考えると，Eckart-Young-Mirskyの定理(単一の行列を低ランク行列で近似する際，最適な近似誤差はその行列の$(k+1)$番目の特異値で与えられるという定理)との関連があります．この補題は1つの行列に対して扱っていた古典的な定理の拡張と言えます．各層でのLoRAのランクが$R$以下という制約があり，表現力が低そうに見えていたとしても，これらの行列の積をとると，全体の表現力を示すランクが上がります．

# LoRAを用いたFNNの表現力
## 問題設定
以下のようにネットワークの構造，問題設定を定義します．

|                   | 記法                                                                                                         | 説明                            |
| :---------------- | :--------------------------------------------------------------------------------------------------------- | :---------------------------- |
| **基本構造**          | $\mathrm{FNN}_{L,D}$                                                                                       | $L$層，幅$D$の全結合ReLUニューラルネットワーク． |
| **重み行列とバイアスベクトル** | $\boldsymbol{W}_l \in \mathbb{R}^{D \times D}$, $\boldsymbol{b}_l \in \mathbb{R}^{D}$                      |                               |
| **ターゲットモデル**      | $\bar{f} := \mathrm{FNN}_{\bar{L}, D}(\cdot ; \overline{\boldsymbol{W}}_l, \overline{\boldsymbol{b}}_l)$   | 層数$\bar{L}$                   |
| **凍結モデル**         | $f_0 := \mathrm{FNN}_{L, D}(\cdot ; \boldsymbol{W}_l, \boldsymbol{b}_l)$                                   | 層数$L$ (ただし$L \ge \bar{L}$)    |
| **適応モデル**         | $f := \mathrm{FNN}_{L, D}(\cdot ; \boldsymbol{W}_l + \Delta \boldsymbol{W}_l, \widehat{\boldsymbol{b}}_l)$ |                               |

## 単層ReLU FNN近似
ここでは，適応モデルが，ReLU$\left(\overline{\boldsymbol{W}}_{1} \cdot+\overline{\boldsymbol{b}}_{1}\right)$を近似できるようなLoRAアダプタ$\left(\Delta \boldsymbol{W}_{l}\right)_{l=1}^{L}$とバイアスベクトル$\left(\widehat{\boldsymbol{b}}_{l}\right)_{l=1}^{L}$を見つけることを目指します．

単層のReLU FNNで表現されるターゲットモデルは以下のような構造を持っています．
![](https://cdn.mathpix.com/snip/images/zlPsVHCXecWvQdoJ1TjXVAzyBiCjnQXUwZMfHOyATYk.original.fullsize.png)
一方で適応モデルは以下のような構造を持っています．
![](https://cdn.mathpix.com/snip/images/Nyz4qlI2eiZegls51p-bjEkgZfy2f3kpGq5CkAGFsl8.original.fullsize.png)
多層なので，出力が入れ子になっています．つまり，出力は以下のようになります．

$$
\operatorname{ReLU}\left(\left(\boldsymbol{W}_{L}+\Delta \boldsymbol{W}_{L}\right) \cdot \operatorname{ReLU}\left(\left(\boldsymbol{W}_{L-1}+\Delta \boldsymbol{W}_{L-1}\right) \cdot \operatorname{ReLU}(\cdots)+\widehat{\boldsymbol{b}}_{L-1}\right)+\widehat{\boldsymbol{b}}_{L}\right)
$$
多層構造はReLUの非線形性もあって扱いにくいです．そこで，線形化と重み行列を調整を行います．

**線形化**
LoRAで適応させるモデル（$L$層ある）の,出力層以外のすべての層（$L-1$ 層）に注目します．これらの層のバイアス項（$\widehat{\boldsymbol{b}}_1, \ldots, \widehat{\boldsymbol{b}}_{L-1}$）の値を意図的に非常に大きくします．

ReLU関数は，入力が0以下なら0を，0より大きければ入力をそのまま通します．バイアス項を非常に大きくすると，その層の計算結果（重み行列との積 + バイアス）は，入力がどんな値であってもほぼ確実に正の大きな値になります．
つまり，ReLU関数は入力をそのまま通すだけになり，実質的に何もしない恒等関数になります．よって，最後の層以外のの非線形性が消えて，線形な変換が連続しているのと同じ状態になります．

その結果，適応モデル全体としては，「入力 → (最初の$L-1$層がまとまった1つの線形変換) → $L$層のReLUとバイアス → 出力」という形になり，実質的に1層のReLU FNNと等価と見なせるようになります．

**重み行列の調整**
適応モデルの最後の層のバイアスを，ターゲットモデル のバイアス項と一致するように調整します．次に，適応モデルのLoRA適用後行列を，$\bar{f}$ の重み行列と一致するように，LoRAアダプタ $\Delta \boldsymbol{W}_l$ を調整します．

## 補題2(近似誤差の上界)
* 誤差行列$\boldsymbol{E}:=\overline{\boldsymbol{W}}_{1}-\prod_{l=1}^{L} \boldsymbol{W}_{l}$，
* $R_{\boldsymbol{E}}=\operatorname{rank}(\boldsymbol{E})$
* LoRAランク：$R \in[D]$
* $\mathbf{x}$：有界サポート$\mathcal{X}$を持つ分布からサンプリングされた確率変数の実現値
* $\Sigma=\mathbb{E} \mathbf{x x}^{\top}$
として，重み行列$\boldsymbol{W}_{1}, \ldots, \boldsymbol{W}_{L} \in \mathbb{R}^{D \times D}$および$\prod_{l=1}^{L} \boldsymbol{W}_{l}+\mathrm{LR}_{r}(\boldsymbol{E})$がすべての$r \leq R(L-1)$に対して正則であると仮定します．

ランク$R$以下の行列$\Delta \boldsymbol{W}_{1}, \ldots, \Delta \boldsymbol{W}_{L} \in \mathbb{R}^{D \times D}$とバイアスベクトル$\widehat{\boldsymbol{b}}_{1}, \ldots, \widehat{\boldsymbol{b}}_{L} \in \mathbb{R}^{D}$が存在し，期待二乗誤差は次のように有界となります．

$$
\mathbb{E}\|f(\mathbf{x})-\bar{f}(\mathbf{x})\|_{2}^{2} \leq\|\Sigma\|_{F} \sigma_{R L+1}^{2}(\boldsymbol{E})
$$

さらに，$R \geq\left\lceil\frac{R_{E}}{L}\right\rceil$の場合，すべての$\boldsymbol{x} \in \mathcal{X}$に対して$f(\boldsymbol{x})=\bar{f}(\boldsymbol{x})$となります．

## 多層ReLU FNN近似
前の節では，多層のモデルで単層のモデルを近似しました．今回のページでは多層のモデルで多層のモデルを近似します．特に，モデル分割という技術を用いて，各層ごとの近似問題としてこの問題に対処します．

## 分割

### 分割の例
$\bar{L}=2$かつ$L=4$の場合を考えます．2層のターゲットモデル$\bar{f}$を2つの1層ReLU FNNの合成を考えます．4層の適応モデル$f$をそれぞれ2層からなる2つのサブモデルに分割します．ターゲットモデルの各層に対して，2つの層を近似に使用することになります．この問題は，補題2で扱った1層FNN近似問題に帰着します．
![](https://cdn.mathpix.com/snip/images/5AmCPIaG8AYut16H_uNyWpfjtYpiRrXaB3NDY5HbS2U.original.fullsize.png)
では，どのように各層を分割するかを検討していきます．いかにいくつかの定義を書きます．
* $\mathcal{P} = \{P_1, P_2, \ldots, P_{\bar{L}}\}$：分割
* $\bar{L}$ : $\bar{f}$ の層の数
先ほどの例で行くと，分割 $\mathcal{P} = \{P_1, P_2\}$ で，$P_1 = \{1, 2\}$，$P_2 = \{3, 4\}$ です．
### 均一分割
次に，モデルの層をできるだけ均等なサイズでグループ分けする均一分割を導入します．
* $M = \lfloor L / \bar{L} \rfloor$：$L$ を目標モデルの層数 $\bar{L}$ で割ったもの（小数点以下切り捨て）

$P_1^{\mathrm{u}} = \{1, \ldots, M\}$ (最初の$M$層)，$P_2^{\mathrm{u}} = \{M+1, \ldots, 2M\}$ (次の$M$層)というように，適応モデルの層を前から順番に $M$ 層ずつ区切ってグループを作っていきます．

****
次に，近似の定理のための仮定と補題を示します．
### 仮定1(正則性)
LoRAランク$R \in[D]$に対して$\left(\boldsymbol{W}_{l}\right)_{l=1}^{L}$と$\left(\prod_{l \in P_{i}^{u}} \boldsymbol{W}_{l}\right)+\operatorname{LR}_{r}\left(\overline{\boldsymbol{W}}_{i}-\prod_{l \in P_{i}^{u}} \boldsymbol{W}_{l}\right)$は，すべての$r \leq R(M-1)$と$i \in[\bar{L}]$に対して正則です．

### 補題3(仮定の成立)
$\left(\overline{\boldsymbol{W}}_{l}\right)_{l=1}^{L},\left(\boldsymbol{W}_{l}\right)_{l=1}^{L} \in \mathbb{R}^{D \times D}$を，その要素が任意の連続分布から独立に抽出される行列とします．そのとき，確率1で，仮定1はすべての$R \in[D]$に対して成り立ちます．
* 重み行列が分布からサンプルされるなんて言う状況ある？

## 定理3
仮定1の下で，LoRAランク$R \geq\left\lceil\max _{i \in[\bar{L}]} \operatorname{rank}\left(\overline{\boldsymbol{W}}_{i}-\prod_{l \in P_{i}^{u}} \boldsymbol{W}_{l}\right) / M\right\rceil$であれば，ランク$R$以下の行列$\Delta \boldsymbol{W}_{1}, \ldots, \Delta \boldsymbol{W}_{L} \in \mathbb{R}^{D \times D}$とバイアスベクトル$\widehat{\boldsymbol{b}}_{1}, \ldots, \widehat{\boldsymbol{b}}_{L} \in \mathbb{R}^{D}$が存在し，$f$は$\bar{f}$を正確に近似できます．すなわち，
$$
f(\boldsymbol{x})=\bar{f}(\boldsymbol{x}), \forall \boldsymbol{x} \in \mathcal{X}
$$です．


## 定理3の証明
TODO

## 系4
$\left(\overline{\boldsymbol{W}}_{l}\right)_{l=1}^{L},\left(\boldsymbol{W}_{l}\right)_{l=1}^{L}$の要素は，任意の連続分布から独立に抽出されると仮定します．$R \geq D / M$の場合，確率1で，ランク$R$以下の行列$\Delta \boldsymbol{W}_{1}, \ldots, \Delta \boldsymbol{W}_{L} \in \mathbb{R}^{D \times D}$とバイアスベクトル$\widehat{\boldsymbol{b}}_{1}, \ldots, \widehat{\boldsymbol{b}}_{L} \in \mathbb{R}^{D}$が存在し，低ランク適応モデル$f$は$\mathcal{X}$上でターゲットモデル$\bar{f}$を正確に近似できます．すなわち，
$$
f(\boldsymbol{x})=\bar{f}(\boldsymbol{x}), \forall \boldsymbol{x} \in \mathcal{X}
$$です．
$L \gg \bar{L}$の場合を考えてみます．ここでは必要なLoRAランクは十分に小さく，ランク$R$行列の次元は約$2 R D$です．つまり，系は．LoRAは$2 R D L \geq 2 D^{2} L / M \approx 2 D^{2} \bar{L}$個の学習パラメータがあれば，ランダムなFNNで $\bar{f}$ に適応できるということを示しています．つまり，定数係数の二倍程度しか要求しません．とても効率がいいですね．


## 定理5(ランクが足りないアダプタでの近似誤差の上界)

$i$番目の層の近似誤差を$E_{i}=\sigma_{R M+1}\left(\overline{\boldsymbol{W}}_{i}-\prod_{l \in P_{i}^{u}} \boldsymbol{W}_{l}\right)$と定義し，パラメータと入力の大きさを
$$
\beta \quad:=\max _{i \in[\bar{L}]}\left(\sqrt{\|\Sigma\|_{F}} \prod_{j=1}^{i}\left\|\overline{\boldsymbol{W}}_{j}\right\|_{F}+\sum_{j=1}^{i} \prod_{k=j+1}^{i-1}\left\|\overline{\boldsymbol{W}}_{k}\right\|_{F}\left\|\overline{\boldsymbol{b}}_{j}\right\|_{2}\right) \bigvee \sqrt{\|\Sigma\|_{F}}
$$と定義します．仮定1の下で，ランク$R$以下の行列$\left(\Delta \boldsymbol{W}_{l}\right)_{l=1}^{L}$（$\Delta \boldsymbol{W}_{l} \in \mathbb{R}^{D \times D}$）とバイアスベクトル$\left(\widehat{\boldsymbol{b}}_{l}\right)_{l=1}^{L}$（$\widehat{\boldsymbol{b}}_{l} \in \mathbb{R}^{D}$）が存在し，$\mathbb{E} \mathbf{x x}^{\top}=\Sigma$を満たす入力$\mathbf{x} \in \mathcal{X}$に対して，
$$
\mathbb{E}\|f(\mathbf{x})-\bar{f}(\mathbf{x})\|_{2} \leq \beta \sum_{i=1}^{\bar{L}} \max _{k \in[\bar{L}]}\left(\left\|\overline{\boldsymbol{W}}_{k}\right\|_{F}+E_{k}\right)^{\bar{L}-i} E_{i} .
$$
が成り立ちます．

一般分割を用いた場合の検証結果は長いのでパスします．
補題4 (最終層のチューニングの限界):ランダムな初期層を持つモデルは，最終層をどれだけ調整しても単純なターゲットさえ完全再現できないです．一方でLoRAはランダムモデルでも少ないパラメータでターゲットを再現できます．

# Transformerの表現力とLoRA
以下のように問題設定を考えます．
はい，承知いたしました．Transformerネットワーク (TFN) におけるLoRAの問題設定の主要な要素を表形式でまとめます．

**TFNにおけるLoRAの問題設定の概要**

| 名称                  | 記法                                                                                                                   | 説明                                                                       |
| :------------------ | :------------------------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------- |
| **基本構造**            | $\operatorname{TFN}_{L, D}$                                                                                          | $L$個のTransformerブロックと1つの出力層から構成されるTransformerネットワーク．埋め込み次元は$D$           |
|                     | $\boldsymbol{W}_{o} \in \mathbb{R}^{D \times D}$                                                                     | 出力層の重み行列                                                                 |
| **Transformerブロック** | $L$個                                                                                                                 | 各ブロックはself-attention層とフィードフォワード層を含む．                                     |
| **self-attention層** | $H$ヘッド                                                                                                               | マルチヘッドself-attention機構                                                   |
|                     | $(\boldsymbol{W}_{Q l}^{h}, \boldsymbol{W}_{K l}^{h}, \boldsymbol{W}_{V l}^{h}, \boldsymbol{W}_{O l}^{h})_{h=1}^{H}$ | 各ヘッド$h$，各ブロック$l$におけるクエリ，キー，バリュー，出力射影の重み行列．すべて $\mathbb{R}^{D \times D}$． |
| **フィードフォワード層**      |                                                                                                                      | トークンごとの2層の全結合ネットワーク．                                                     |
|                     | $\boldsymbol{W}_{1 l}, \boldsymbol{W}_{2 l} \in \mathbb{R}^{D \times D}$                                             | フィードフォワード層の1層目と2層目の重み行列．                                                 |
|                     | $\boldsymbol{b}_{1 l}, \boldsymbol{b}_{2 l} \in \mathbb{R}^{D}$                                                      | フィードフォワード層の1層目と2層目のバイアスベクトル．                                             |
| **ターゲットモデル**        | $\bar{f}$ (パラメータ例: $\overline{\boldsymbol{W}}_{1l}, \overline{\boldsymbol{W}}_{Ql}^{h}$)                             | 凍結モデルと同じ$L$個のTransformerブロックを持つと仮定．                                      |
| **凍結モデル**           | $f_0$ (パラメータ例: $\boldsymbol{W}_{1l}, \boldsymbol{W}_{Ql}^{h}$)                                                       | ターゲットモデルと同じ$L$個のTransformerブロックを持つ．                                      |
| **LoRAアダプタ**        | $\Delta \boldsymbol{W}_{\cdot l}^{\cdot}$ (例: $\Delta \boldsymbol{W}_{Ql}^{h}$)                                      | 各凍結重み行列に追加される低ランクの更新行列．ランク制約あり．                                          |
| **更新バイアス**          | $\widehat{\boldsymbol{b}}_{\cdot l}$ (例: $\widehat{\boldsymbol{b}}_{1l}$)                                            | 適応後のモデルの（更新された）バイアスベクトル．                                                 |
| **構造上の仮定**          | スキップ接続なし                                                                                                             |                                                                          |
|                     | 層数 $L$ は共通                                                                                                           | 凍結モデルとターゲットモデルのTransformerブロック数は同じ $L$ とする．                              |

### 仮定4(正則性)
仮定4（正則性）．固定された$R \in[D]$に対して，ターゲットモデルと凍結モデルの両方のすべての重み行列，およびすべての$r \in[R]$に対する次の行列，

$$
\begin{gathered}
\boldsymbol{W}_{K l}^{h \top} \boldsymbol{W}_{Q l}^{h}+\mathrm{LR}_{r}\left(\overline{\boldsymbol{W}}_{K l}^{h \top} \overline{\boldsymbol{W}}_{Q l}^{h}-\boldsymbol{W}_{K l}^{h \top} \boldsymbol{W}_{Q l}^{h}\right), \text { for all } h \in[H] \text { and } l=1, \\
\boldsymbol{W}_{K l}^{h \top} \boldsymbol{W}_{Q l}^{h}+\mathrm{LR}_{r}\left(\boldsymbol{W}_{2, l-1}^{-1 \top} \overline{\boldsymbol{W}}_{2, l-1}^{\top} \overline{\boldsymbol{W}}_{K l}^{h \top} \overline{\boldsymbol{W}}_{Q l}^{h} \overline{\boldsymbol{W}}_{2, l-1} \boldsymbol{W}_{2, l-1}^{-1}-\boldsymbol{W}_{K l}^{h \top} \boldsymbol{W}_{Q l}^{h}\right), \text { for all } h \in[H] \text { and } l \in[L] \backslash\{1\}, \\
\boldsymbol{W}_{O l}^{h} \boldsymbol{W}_{V l}^{h}+\mathrm{LR}_{r}\left(\boldsymbol{W}_{1 l}^{-1} \overline{\boldsymbol{W}}_{1 l} \overline{\boldsymbol{W}}_{O l}^{h} \overline{\boldsymbol{W}}_{V l}^{h}-\boldsymbol{W}_{O l}^{h} \boldsymbol{W}_{V l}^{h}\right), \text { for all } h \in[H] \text { and } l=1, \\
\boldsymbol{W}_{O l}^{h} \boldsymbol{W}_{V l}^{h}+\mathrm{LR}_{r}\left(\boldsymbol{W}_{1 l}^{-1} \overline{\boldsymbol{W}}_{1 l} \overline{\boldsymbol{W}}_{O l}^{h} \overline{\boldsymbol{W}}_{V l}^{h} \overline{\boldsymbol{W}}_{2, l-1} \boldsymbol{W}_{2, l-1}^{-1}-\boldsymbol{W}_{O l}^{h} \boldsymbol{W}_{V l}^{h}\right), \text { for all } h \in[H] \text { and } l \in[L] \backslash\{1\}, \\
\boldsymbol{W}_{o} \boldsymbol{W}_{2 L}+\mathrm{LR}_{r}\left(\overline{\boldsymbol{W}}_{o} \overline{\boldsymbol{W}}_{2 L}-\boldsymbol{W}_{o} \boldsymbol{W}_{2 L}\right),
\end{gathered}
$$

は正則とします．
## 定理7(self-attention層にLoRA適用をすることによるターゲットモデルの正確な近似)
仮定4が成り立つとします．
* $G_{i}$：$i$番目のTransformerブロック（$i \in[L]$）または出力層（$i=L+1$）に対するランクベースの機能ギャップ(LoRAを使って凍結モデルをターゲットモデルに近づけようとするとき，各Transformerブロック（または出力層）において，どれだけ「ランク」の観点から機能的なズレがあるかを示す指標)
$$
G_{i}= \begin{cases}\max _{h}\left(\operatorname{rank}\left(\overline{\boldsymbol{W}}_{K i}^{h \top} \overline{\boldsymbol{W}}_{Q i}^{h}-\boldsymbol{W}_{K i}^{h \top} \boldsymbol{W}_{Q i}^{h}\right)\right) \vee \max _{h}\left(\operatorname{rank}\left(\overline{\boldsymbol{W}}_{1 i} \overline{\boldsymbol{W}}_{O i}^{h} \overline{\boldsymbol{W}}_{V i}^{h}-\boldsymbol{W}_{1 i} \boldsymbol{W}_{O i}^{h} \boldsymbol{W}_{V i}^{h}\right)\right), i=1,  \tag{23}\\ \max _{h}\left(\operatorname{rank}\left(\overline{\boldsymbol{W}}_{2, i-1}^{\top} \overline{\boldsymbol{W}}_{K i}^{h \top} \overline{\boldsymbol{W}}_{Q i}^{h} \overline{\boldsymbol{W}}_{2, i-1}-\boldsymbol{W}_{2, i-1}^{\top} \boldsymbol{W}_{K i}^{h \top} \boldsymbol{W}_{Q i}^{h} \boldsymbol{W}_{2, i-1}\right)\right) \\ \vee \max _{h}\left(\operatorname{rank}\left(\overline{\boldsymbol{W}}_{1 i} \overline{\boldsymbol{W}}_{O i}^{h} \overline{\boldsymbol{W}}_{V i}^{h} \overline{\boldsymbol{W}}_{2, i-1}-\boldsymbol{W}_{1 i} \boldsymbol{W}_{O i}^{h} \boldsymbol{W}_{V i}^{h} \boldsymbol{W}_{2, i-1}\right)\right), & 2 \leq i \leq L, \\ \operatorname{rank}\left(\overline{\boldsymbol{W}}_{o} \overline{\boldsymbol{W}}_{2 L}-\boldsymbol{W}_{o} \boldsymbol{W}_{2 L}\right), & i=L+1 .\end{cases}
$$
$R \geq \max _{i \in[L+1]}\left\lceil\frac{G_{i}}{2}\right\rceil$であれば，ランクが$R \in[D]$より低い低ランクアダプタ$\left(\left(\Delta \boldsymbol{W}_{K l}^{h}, \Delta \boldsymbol{W}_{Q l}^{h}, \Delta \boldsymbol{W}_{V l}^{h}, \Delta \boldsymbol{W}_{O l}^{h}\right)_{h=1}^{H}\right)_{l=1}^{L}, \Delta \boldsymbol{W}_{2 L}, \Delta \boldsymbol{W}_{o}$（他の低ランクアダプタは$\boldsymbol{O}$に設定）と更新されたバイアスベクトル$\left(\widehat{\boldsymbol{b}}_{1 l}, \widehat{\boldsymbol{b}}_{2 l}\right)_{l=1}^{L}$が存在し，任意の$\boldsymbol{X} \in \mathbb{R}^{D \times N}$に対して，適応モデル$f$はターゲットモデル$\bar{f}$を正確に近似します．すなわち，

$$
f(\boldsymbol{X})=\bar{f}(\boldsymbol{X})
$$です．

## 定理7の証明
この証明の基本的な戦略は，ターゲットモデル $\bar{f}$ と適応モデル $f$ の各Transformerブロックの内部状態を順番に一致させていくことです．具体的には，$l$ 番目のTransformerブロックにおいて，アテンション層の出力と，それに続く最初のフィードフォワード層 (FFN1) のReLU活性化後の出力が，両モデルで一致するようにLoRAアダプタとバイアスを構成します．これを全てのブロックで達成し，最後に全体の出力層を一致させます．

**記法の定義:**
*   $\overline{\boldsymbol{Z}}_{l-1}$: ターゲットモデルにおける $l$ 番目のTransformerブロックへの入力．
*   $\widehat{\boldsymbol{Z}}_{l-1}$: 適応モデルにおける $l$ 番目のTransformerブロックへの入力．($\overline{\boldsymbol{Z}}_0 = \widehat{\boldsymbol{Z}}_0 = \boldsymbol{X}$)
*   $\overline{\boldsymbol{A}}_l(\overline{\boldsymbol{Z}}_{l-1})$: ターゲットモデルの $l$ 番目のブロックのアテンション層の出力．
    $\overline{\boldsymbol{A}}_l(\overline{\boldsymbol{Z}}_{l-1}) = \sum_{h=1}^{H} \overline{\boldsymbol{W}}_{O l}^{h} \overline{\boldsymbol{W}}_{V l}^{h} \overline{\boldsymbol{Z}}_{l-1} \operatorname{softmax}\left(\frac{\overline{\boldsymbol{Z}}_{l-1}^{\top} \overline{\boldsymbol{W}}_{K l}^{h \top} \overline{\boldsymbol{W}}_{Q l}^{h} \overline{\boldsymbol{Z}}_{l-1}}{\sqrt{d_k}}\right)$
    (簡単のため，スケーリングファクタ $\sqrt{d_k}$ は省略されているか，重みに含まれていると解釈します．論文では明示されていませんが，softmaxの引数に影響します．)
*   $\widehat{\boldsymbol{A}}_l(\widehat{\boldsymbol{Z}}_{l-1})$: 適応モデルの $l$ 番目のブロックのアテンション層の出力．LoRAアダプタが適用される．
    $\widehat{\boldsymbol{A}}_l(\widehat{\boldsymbol{Z}}_{l-1}) = \sum_{h=1}^{H} (\boldsymbol{W}_{O l}^{h}+\Delta \boldsymbol{W}_{O l}^{h}) (\boldsymbol{W}_{V l}^{h}+\Delta \boldsymbol{W}_{V l}^{h}) \widehat{\boldsymbol{Z}}_{l-1} \operatorname{softmax}\left(\frac{\widehat{\boldsymbol{Z}}_{l-1}^{\top} (\boldsymbol{W}_{K l}^{h}+\Delta \boldsymbol{W}_{K l}^{h})^{\top} (\boldsymbol{W}_{Q l}^{h}+\Delta \boldsymbol{W}_{Q l}^{h}) \widehat{\boldsymbol{Z}}_{l-1}}{\sqrt{d_k}}\right)$
*   $\overline{\boldsymbol{H}}_l$: ターゲットモデルの $l$ 番目のブロックのFFN1のReLU後の出力．
    $\overline{\boldsymbol{H}}_l = \operatorname{ReLU}(\overline{\boldsymbol{W}}_{1l} \overline{\boldsymbol{A}}_l(\overline{\boldsymbol{Z}}_{l-1}) + \overline{\boldsymbol{b}}_{1l} \mathbf{1}_N^{\top})$
*   $\widehat{\boldsymbol{H}}_l$: 適応モデルの $l$ 番目のブロックのFFN1のReLU後の出力．
    $\widehat{\boldsymbol{H}}_l = \operatorname{ReLU}((\boldsymbol{W}_{1l}+\Delta \boldsymbol{W}_{1l}) \widehat{\boldsymbol{A}}_l(\widehat{\boldsymbol{Z}}_{l-1}) + \widehat{\boldsymbol{b}}_{1l} \mathbf{1}_N^{\top})$
    (論文では $\Delta \boldsymbol{W}_{1l} = \boldsymbol{O}$ としているので，$\widehat{\boldsymbol{H}}_l = \operatorname{ReLU}(\boldsymbol{W}_{1l} \widehat{\boldsymbol{A}}_l(\widehat{\boldsymbol{Z}}_{l-1}) + \widehat{\boldsymbol{b}}_{1l} \mathbf{1}_N^{\top})$)
*   $\overline{\boldsymbol{Z}}_l$: ターゲットモデルの $l$ 番目のブロックの最終出力．
    $\overline{\boldsymbol{Z}}_l = \overline{\boldsymbol{W}}_{2l} \overline{\boldsymbol{H}}_l + \overline{\boldsymbol{b}}_{2l} \mathbf{1}_N^{\top}$
*   $\widehat{\boldsymbol{Z}}_l$: 適応モデルの $l$ 番目のブロックの最終出力．
    $\widehat{\boldsymbol{Z}}_l = (\boldsymbol{W}_{2l}+\Delta \boldsymbol{W}_{2l}) \widehat{\boldsymbol{H}}_l + \widehat{\boldsymbol{b}}_{2l} \mathbf{1}_N^{\top}$
    (論文では，$l < L$ のとき $\Delta \boldsymbol{W}_{2l} = \boldsymbol{O}$ とし，$l=L$ のときのみ $\Delta \boldsymbol{W}_{2L}$ を考慮する．)

$l=1, \ldots, L$ の各ブロックについて，$\widehat{\boldsymbol{H}}_l = \overline{\boldsymbol{H}}_l$ が成り立つようにLoRAアダプタとバイアスを構成することを目指します．
さらに，帰納法の仮定として，$l-1$ 番目のブロックまでで $\widehat{\boldsymbol{Z}}_{l-1}$ と $\overline{\boldsymbol{Z}}_{l-1}$ の間に特定の線形関係が成り立つようにします．

**基底ステップ ($l=1$):**
このとき，$\widehat{\boldsymbol{Z}}_0 = \overline{\boldsymbol{Z}}_0 = \boldsymbol{X}$ です．
目標は $\widehat{\boldsymbol{H}}_1 = \overline{\boldsymbol{H}}_1$ を達成することです．これは，以下の2つの条件が満たされればよいことになります．
1.  **アテンション層の出力の一致の前の部分:** softmaxの引数が一致し，かつ，softmax後の値と線形変換の結果が一致する．
    *   **softmaxの引数の一致:** 各ヘッド $h$ について，
        $(\boldsymbol{W}_{K 1}^{h}+\Delta \boldsymbol{W}_{K 1}^{h})^{\top} (\boldsymbol{W}_{Q 1}^{h}+\Delta \boldsymbol{W}_{Q 1}^{h}) = \overline{\boldsymbol{W}}_{K 1}^{h \top} \overline{\boldsymbol{W}}_{Q 1}^{h}$
        これを達成するために，補題7を $(\boldsymbol{W}_{K 1}^{h})^{\top}$ と $(\boldsymbol{W}_{Q 1}^{h})$ に適用します（実際には $(\boldsymbol{W}_{K 1}^{h}+\Delta \boldsymbol{W}_{K 1}^{h})^{\top}$ と $(\boldsymbol{W}_{Q 1}^{h}+\Delta \boldsymbol{W}_{Q 1}^{h})$ の積をターゲットに合わせるので，2つの行列の積の近似）．
        誤差行列は $\boldsymbol{E}_{KQ,1}^h = \overline{\boldsymbol{W}}_{K 1}^{h \top} \overline{\boldsymbol{W}}_{Q 1}^{h} - \boldsymbol{W}_{K 1}^{h \top} \boldsymbol{W}_{Q 1}^{h}$．
        LoRAランク $R$ が $R \geq \lceil \operatorname{rank}(\boldsymbol{E}_{KQ,1}^h) / 2 \rceil$ であれば，$\Delta \boldsymbol{W}_{K 1}^h$ と $\Delta \boldsymbol{W}_{Q 1}^h$ を構成してこれを達成できます（例えば，一方を固定して他方を補題7で調整，あるいは両者を調整）．定理7の条件 $R \geq \lceil G_1/2 \rceil$ はこれを保証します．
    *   **アテンション層の出力の線形部分の一致:** softmaxの出力が同じであると仮定すると（引数が同じなので），次に $(\boldsymbol{W}_{O 1}^{h}+\Delta \boldsymbol{W}_{O 1}^{h}) (\boldsymbol{W}_{V 1}^{h}+\Delta \boldsymbol{W}_{V 1}^{h})$ が $\overline{\boldsymbol{W}}_{O 1}^{h} \overline{\boldsymbol{W}}_{V 1}^{h}$ と一致する必要があります．
        誤差行列は $\boldsymbol{E}_{OV,1}^h = \overline{\boldsymbol{W}}_{O 1}^{h} \overline{\boldsymbol{W}}_{V 1}^{h} - \boldsymbol{W}_{O 1}^{h} \boldsymbol{W}_{V 1}^{h}$．
        同様に，LoRAランク $R$ が $R \geq \lceil \operatorname{rank}(\boldsymbol{E}_{OV,1}^h) / 2 \rceil$ であれば，$\Delta \boldsymbol{W}_{O 1}^h$ と $\Delta \boldsymbol{W}_{V 1}^h$ を構成してこれを達成できます．
    これにより，$\widehat{\boldsymbol{A}}_1(\boldsymbol{X}) = \overline{\boldsymbol{A}}_1(\boldsymbol{X})$ となります．

2.  **FFN1の出力の一致:**
    論文では $\Delta \boldsymbol{W}_{1l} = \boldsymbol{O}$ としているので，$\boldsymbol{W}_{11} \widehat{\boldsymbol{A}}_1(\boldsymbol{X}) + \widehat{\boldsymbol{b}}_{11} \mathbf{1}_N^{\top}$ と $\overline{\boldsymbol{W}}_{11} \overline{\boldsymbol{A}}_1(\boldsymbol{X}) + \overline{\boldsymbol{b}}_{11} \mathbf{1}_N^{\top}$ のReLU適用の結果を一致させます．
    $\widehat{\boldsymbol{A}}_1 = \overline{\boldsymbol{A}}_1$ は達成済みです．
    もし $\boldsymbol{W}_{11} = \overline{\boldsymbol{W}}_{11}$ であれば，$\widehat{\boldsymbol{b}}_{11} = \overline{\boldsymbol{b}}_{11}$ とすれば $\widehat{\boldsymbol{H}}_1 = \overline{\boldsymbol{H}}_1$ となります．
    しかし，一般には $\boldsymbol{W}_{11} \neq \overline{\boldsymbol{W}}_{11}$ です．証明の本文では，$\boldsymbol{W}_{O l}^{h} \boldsymbol{W}_{V l}^{h}$ の部分を $\boldsymbol{W}_{1l}^{-1} \overline{\boldsymbol{W}}_{1l} \overline{\boldsymbol{W}}_{O l}^{h} \overline{\boldsymbol{W}}_{V l}^{h}$ に合わせるように $\Delta \boldsymbol{W}_{O l}^{h}, \Delta \boldsymbol{W}_{V l}^{h}$ を調整しています．これにより，FFN1の入力が $\boldsymbol{W}_{1l} (\boldsymbol{W}_{1l}^{-1} \overline{\boldsymbol{W}}_{1l} \overline{\boldsymbol{A}}_l(\cdot)) = \overline{\boldsymbol{W}}_{1l} \overline{\boldsymbol{A}}_l(\cdot)$ となり，ターゲットと一致します．
    このためには，誤差行列 $\boldsymbol{E}_{OV',1}^h = \boldsymbol{W}_{11}^{-1} \overline{\boldsymbol{W}}_{11} \overline{\boldsymbol{W}}_{O 1}^{h} \overline{\boldsymbol{W}}_{V 1}^{h} - \boldsymbol{W}_{O 1}^{h} \boldsymbol{W}_{V 1}^{h}$ に対して，ランク $R$ が $R \geq \lceil \operatorname{rank}(\boldsymbol{E}_{OV',1}^h) / 2 \rceil$ を満たすように $\Delta \boldsymbol{W}_{O 1}^h, \Delta \boldsymbol{W}_{V 1}^h$ を構成します．定理7の条件 $R \geq \lceil G_1/2 \rceil$ はこれを保証します（$G_1$ の定義に $\operatorname{rank}(\overline{\boldsymbol{W}}_{1i} \overline{\boldsymbol{W}}_{O i}^{h} \overline{\boldsymbol{W}}_{V i}^{h}-\boldsymbol{W}_{1 i} \boldsymbol{W}_{O i}^{h} \boldsymbol{W}_{V i}^{h})$ が含まれており，これは $\boldsymbol{W}_{1i}(\boldsymbol{W}_{1i}^{-1} \overline{\boldsymbol{W}}_{1i} \overline{\boldsymbol{W}}_{O i}^{h} \overline{\boldsymbol{W}}_{V i}^{h} - \boldsymbol{W}_{O i}^{h} \boldsymbol{W}_{V i}^{h})$ のランクと同じです）．
    そして，$\widehat{\boldsymbol{b}}_{11} = \overline{\boldsymbol{b}}_{11}$ と設定します．
    これにより，$\widehat{\boldsymbol{H}}_1 = \overline{\boldsymbol{H}}_1$ が達成されます．

最後に，$l=1$ のブロックの出力 $\widehat{\boldsymbol{Z}}_1$ と $\overline{\boldsymbol{Z}}_1$ の関係を考えます．
論文では $\Delta \boldsymbol{W}_{2l}=\boldsymbol{O}$ for $l<L$ としています．
$\widehat{\boldsymbol{Z}}_1 = \boldsymbol{W}_{21} \widehat{\boldsymbol{H}}_1 + \widehat{\boldsymbol{b}}_{21} \mathbf{1}_N^{\top}$
$\overline{\boldsymbol{Z}}_1 = \overline{\boldsymbol{W}}_{21} \overline{\boldsymbol{H}}_1 + \overline{\boldsymbol{b}}_{21} \mathbf{1}_N^{\top}$
$\widehat{\boldsymbol{H}}_1 = \overline{\boldsymbol{H}}_1$ は達成済みなので，
$\widehat{\boldsymbol{b}}_{21} = \boldsymbol{W}_{21} \overline{\boldsymbol{W}}_{21}^{-1} (\overline{\boldsymbol{Z}}_1 - \overline{\boldsymbol{b}}_{21} \mathbf{1}_N^{\top}) - \boldsymbol{W}_{21} \overline{\boldsymbol{H}}_1 + \overline{\boldsymbol{b}}_{21} \mathbf{1}_N^{\top}$
ではなく，論文の証明F.2では，$\widehat{\boldsymbol{b}}_{2,l-1} = \boldsymbol{W}_{2,l-1} \overline{\boldsymbol{W}}_{2,l-1}^{-1} \overline{\boldsymbol{b}}_{2,l-1}$ と設定し，結果として $\widehat{\boldsymbol{Z}}_{l-1} = \boldsymbol{W}_{2,l-1} \overline{\boldsymbol{W}}_{2,l-1}^{-1} \overline{\boldsymbol{Z}}_{l-1}$ という関係を帰納的に維持します．
$l=1$ の場合，$\widehat{\boldsymbol{Z}}_0 = \boldsymbol{X}$，$\overline{\boldsymbol{Z}}_0 = \boldsymbol{X}$ なので，この関係は $\boldsymbol{W}_{20}\overline{\boldsymbol{W}}_{20}^{-1}$ が単位行列でない限り成り立ちません．
この部分は論文の付録の記述と整合させる必要があります．付録の証明では，$\widehat{\boldsymbol{H}}_{l-1} = \overline{\boldsymbol{H}}_{l-1}$ を帰納法の仮定とし，$\widehat{\boldsymbol{Z}}_{l-1} = \boldsymbol{W}_{2,l-1} \overline{\boldsymbol{H}}_{l-1} + \widehat{\boldsymbol{b}}_{2,l-1}$ と $\overline{\boldsymbol{Z}}_{l-1} = \overline{\boldsymbol{W}}_{2,l-1} \overline{\boldsymbol{H}}_{l-1} + \overline{\boldsymbol{b}}_{2,l-1}$ から，
$\widehat{\boldsymbol{b}}_{2,l-1} = \boldsymbol{W}_{2,l-1} \overline{\boldsymbol{W}}_{2,l-1}^{-1} \overline{\boldsymbol{b}}_{2,l-1}$ と設定することで，
$\widehat{\boldsymbol{Z}}_{l-1} - \boldsymbol{W}_{2,l-1} \overline{\boldsymbol{W}}_{2,l-1}^{-1} \overline{\boldsymbol{Z}}_{l-1} = (\boldsymbol{W}_{2,l-1} - \boldsymbol{W}_{2,l-1} \overline{\boldsymbol{W}}_{2,l-1}^{-1} \overline{\boldsymbol{W}}_{2,l-1}) \overline{\boldsymbol{H}}_{l-1} = \boldsymbol{O}$
となり，$\widehat{\boldsymbol{Z}}_{l-1} = \boldsymbol{W}_{2,l-1} \overline{\boldsymbol{W}}_{2,l-1}^{-1} \overline{\boldsymbol{Z}}_{l-1}$ が成り立ちます．
この関係を次のステップの入力として使います．

**帰納ステップ ($l > 1$):**
帰納法の仮定として，$(l-1)$ 番目のブロックまでで $\widehat{\boldsymbol{H}}_{l-1} = \overline{\boldsymbol{H}}_{l-1}$ が達成され，かつ $\widehat{\boldsymbol{Z}}_{l-1} = \boldsymbol{W}_{2,l-1} \overline{\boldsymbol{W}}_{2,l-1}^{-1} \overline{\boldsymbol{Z}}_{l-1}$ が成り立つとします．
目標は $\widehat{\boldsymbol{H}}_l = \overline{\boldsymbol{H}}_l$ を達成することです．
1.  **softmaxの引数の一致:** 各ヘッド $h$ について，
    $(\boldsymbol{W}_{K l}^{h}+\Delta \boldsymbol{W}_{K l}^{h})^{\top} (\boldsymbol{W}_{Q l}^{h}+\Delta \boldsymbol{W}_{Q l}^{h})$ を
    $(\boldsymbol{W}_{2,l-1} \overline{\boldsymbol{W}}_{2,l-1}^{-1})^{\top} \overline{\boldsymbol{W}}_{K l}^{h \top} \overline{\boldsymbol{W}}_{Q l}^{h} (\boldsymbol{W}_{2,l-1} \overline{\boldsymbol{W}}_{2,l-1}^{-1})^{-1 \top}$ のような形に合わせるのではなく，論文では，
    $\widehat{\boldsymbol{Z}}_{l-1}^{\top} \left((\boldsymbol{W}_{K l}^{h}+\Delta \boldsymbol{W}_{K l}^{h})^{\top} (\boldsymbol{W}_{Q l}^{h}+\Delta \boldsymbol{W}_{Q l}^{h})\right) \widehat{\boldsymbol{Z}}_{l-1}$ が
    $\overline{\boldsymbol{Z}}_{l-1}^{\top} \left(\overline{\boldsymbol{W}}_{K l}^{h \top} \overline{\boldsymbol{W}}_{Q l}^{h}\right) \overline{\boldsymbol{Z}}_{l-1}$ と一致するようにします．
    $\widehat{\boldsymbol{Z}}_{l-1} = \boldsymbol{C}_{l-1} \overline{\boldsymbol{Z}}_{l-1}$ （ただし $\boldsymbol{C}_{l-1} = \boldsymbol{W}_{2,l-1} \overline{\boldsymbol{W}}_{2,l-1}^{-1}$）という関係を使うと，
    $\overline{\boldsymbol{Z}}_{l-1}^{\top} \boldsymbol{C}_{l-1}^{\top} \left((\boldsymbol{W}_{K l}^{h}+\Delta \boldsymbol{W}_{K l}^{h})^{\top} (\boldsymbol{W}_{Q l}^{h}+\Delta \boldsymbol{W}_{Q l}^{h})\right) \boldsymbol{C}_{l-1} \overline{\boldsymbol{Z}}_{l-1}$ をターゲットに合わせます．
    よって，$\boldsymbol{C}_{l-1}^{\top} (\boldsymbol{W}_{K l}^{h}+\Delta \boldsymbol{W}_{K l}^{h})^{\top} (\boldsymbol{W}_{Q l}^{h}+\Delta \boldsymbol{W}_{Q l}^{h}) \boldsymbol{C}_{l-1} = \overline{\boldsymbol{W}}_{K l}^{h \top} \overline{\boldsymbol{W}}_{Q l}^{h}$
    つまり，$(\boldsymbol{W}_{K l}^{h}+\Delta \boldsymbol{W}_{K l}^{h})^{\top} (\boldsymbol{W}_{Q l}^{h}+\Delta \boldsymbol{W}_{Q l}^{h}) = \boldsymbol{C}_{l-1}^{-1 \top} \overline{\boldsymbol{W}}_{K l}^{h \top} \overline{\boldsymbol{W}}_{Q l}^{h} \boldsymbol{C}_{l-1}^{-1}$
    誤差行列は $\boldsymbol{E}_{KQ,l}^h = \boldsymbol{C}_{l-1}^{-1 \top} \overline{\boldsymbol{W}}_{K l}^{h \top} \overline{\boldsymbol{W}}_{Q l}^{h} \boldsymbol{C}_{l-1}^{-1} - \boldsymbol{W}_{K l}^{h \top} \boldsymbol{W}_{Q l}^{h}$．
    定理7の $G_i$ の定義内の $\operatorname{rank}(\overline{\boldsymbol{W}}_{2, i-1}^{\top} \overline{\boldsymbol{W}}_{K i}^{h \top} \overline{\boldsymbol{W}}_{Q i}^{h} \overline{\boldsymbol{W}}_{2, i-1}-\boldsymbol{W}_{2, i-1}^{\top} \boldsymbol{W}_{K i}^{h \top} \boldsymbol{W}_{Q i}^{h} \boldsymbol{W}_{2, i-1})$ と比較すると，$\boldsymbol{C}_{l-1}$ の形が少し異なりますが，基本的な考え方は，変換された空間でターゲットに合わせることです．論文の証明の仮定4で使われる行列の形を見ると，
    $\boldsymbol{W}_{K l}^{h \top} \boldsymbol{W}_{Q l}^{h}+\mathrm{LR}_{r}\left(\boldsymbol{W}_{2, l-1}^{-1 \top} \overline{\boldsymbol{W}}_{2, l-1}^{\top} \overline{\boldsymbol{W}}_{K l}^{h \top} \overline{\boldsymbol{W}}_{Q l}^{h} \overline{\boldsymbol{W}}_{2, l-1} \boldsymbol{W}_{2, l-1}^{-1}-\boldsymbol{W}_{K l}^{h \top} \boldsymbol{W}_{Q l}^{h}\right)$
    が正則であるという仮定があります．これは，
    ターゲットを $\boldsymbol{T}_{KQ,l}^h = \boldsymbol{W}_{2, l-1}^{-1 \top} \overline{\boldsymbol{W}}_{2, l-1}^{\top} \overline{\boldsymbol{W}}_{K l}^{h \top} \overline{\boldsymbol{W}}_{Q l}^{h} \overline{\boldsymbol{W}}_{2, l-1} \boldsymbol{W}_{2, l-1}^{-1}$ として，
    $(\boldsymbol{W}_{K l}^{h}+\Delta \boldsymbol{W}_{K l}^{h})^{\top} (\boldsymbol{W}_{Q l}^{h}+\Delta \boldsymbol{W}_{Q l}^{h}) = \boldsymbol{T}_{KQ,l}^h$ を目指します．
    誤差 $\boldsymbol{E}_{KQ,l}^h = \boldsymbol{T}_{KQ,l}^h - \boldsymbol{W}_{K l}^{h \top} \boldsymbol{W}_{Q l}^{h}$ に対して，ランク $R \ge \lceil \operatorname{rank}(\boldsymbol{E}_{KQ,l}^h)/2 \rceil$ であれば達成可能です．定理7の $G_l$ の定義がこれをカバーします．

2.  **アテンション層の出力の線形部分の一致:** 同様に，
    $(\boldsymbol{W}_{O l}^{h}+\Delta \boldsymbol{W}_{O l}^{h}) (\boldsymbol{W}_{V l}^{h}+\Delta \boldsymbol{W}_{V l}^{h})$ を
    $\boldsymbol{T}_{OV,l}^h = \boldsymbol{W}_{1l}^{-1} \overline{\boldsymbol{W}}_{1l} \overline{\boldsymbol{W}}_{O l}^{h} \overline{\boldsymbol{W}}_{V l}^{h} \overline{\boldsymbol{W}}_{2,l-1} \boldsymbol{W}_{2,l-1}^{-1}$ に合わせます．
    誤差 $\boldsymbol{E}_{OV,l}^h = \boldsymbol{T}_{OV,l}^h - \boldsymbol{W}_{O l}^{h} \boldsymbol{W}_{V l}^{h}$ に対して，ランク $R \ge \lceil \operatorname{rank}(\boldsymbol{E}_{OV,l}^h)/2 \rceil$ であれば達成可能です．定理7の $G_l$ の定義がこれをカバーします．
    これにより，$\boldsymbol{W}_{1l} \widehat{\boldsymbol{A}}_l(\widehat{\boldsymbol{Z}}_{l-1}) = \overline{\boldsymbol{W}}_{1l} \overline{\boldsymbol{A}}_l(\overline{\boldsymbol{Z}}_{l-1})$ となります．

3.  **FFN1の出力の一致:** $\widehat{\boldsymbol{b}}_{1l} = \overline{\boldsymbol{b}}_{1l}$ と設定することで，$\widehat{\boldsymbol{H}}_l = \overline{\boldsymbol{H}}_l$ が達成されます．

4.  **次のブロックへの入力の準備:** $l < L$ の場合，$\Delta \boldsymbol{W}_{2l} = \boldsymbol{O}$ とし，$\widehat{\boldsymbol{b}}_{2l} = \boldsymbol{W}_{2l} \overline{\boldsymbol{W}}_{2l}^{-1} \overline{\boldsymbol{b}}_{2l}$ と設定します．これにより，帰納法の仮定で述べた $\widehat{\boldsymbol{Z}}_l = \boldsymbol{W}_{2l} \overline{\boldsymbol{W}}_{2l}^{-1} \overline{\boldsymbol{Z}}_l$ という関係が次のステップでも維持されます．

**最終ステップ ($l=L$ の後，出力層):**
上記の手順により，$l=1, \ldots, L$ の全てのブロックで $\widehat{\boldsymbol{H}}_l = \overline{\boldsymbol{H}}_l$ が達成されています．
次に，最後のブロックの出力 $\widehat{\boldsymbol{Z}}_L$ と，最終的なモデル出力 $f(\boldsymbol{X})$ をターゲットに合わせます．
$\widehat{\boldsymbol{Z}}_L = (\boldsymbol{W}_{2L}+\Delta \boldsymbol{W}_{2L}) \widehat{\boldsymbol{H}}_L + \widehat{\boldsymbol{b}}_{2L} \mathbf{1}_N^{\top}$
$\overline{\boldsymbol{Z}}_L = \overline{\boldsymbol{W}}_{2L} \overline{\boldsymbol{H}}_L + \overline{\boldsymbol{b}}_{2L} \mathbf{1}_N^{\top}$
$\widehat{\boldsymbol{H}}_L = \overline{\boldsymbol{H}}_L$ なので，$\boldsymbol{W}_{2L}+\Delta \boldsymbol{W}_{2L} = \overline{\boldsymbol{W}}_{2L}$ と $\widehat{\boldsymbol{b}}_{2L} = \overline{\boldsymbol{b}}_{2L}$ を達成する必要があります．
$\Delta \boldsymbol{W}_{2L}$ は，誤差 $\overline{\boldsymbol{W}}_{2L} - \boldsymbol{W}_{2L}$ をターゲットとして補題7（この場合は層数が1なので $R \ge \operatorname{rank}(\text{誤差})$）で構成します．$G_{L+1}$ の定義の一部である $\operatorname{rank}(\overline{\boldsymbol{W}}_{o} \overline{\boldsymbol{W}}_{2L}-\boldsymbol{W}_{o} \boldsymbol{W}_{2L})$ は，出力層まで含めた場合の誤差を考慮していますが，ここではまず $\boldsymbol{Z}_L$ を合わせます．
実際には，論文の証明では，出力層の重み $\boldsymbol{W}_o$ と最後のFFN2の重み $\boldsymbol{W}_{2L}$ をまとめて調整します．
最終出力は $f(\boldsymbol{X}) = \operatorname{softmax}((\boldsymbol{W}_o+\Delta \boldsymbol{W}_o) \widehat{\boldsymbol{Z}}_L)$ と $\bar{f}(\boldsymbol{X}) = \operatorname{softmax}(\overline{\boldsymbol{W}}_o \overline{\boldsymbol{Z}}_L)$ です．
softmaxの引数が一致すればよいので，$(\boldsymbol{W}_o+\Delta \boldsymbol{W}_o) \widehat{\boldsymbol{Z}}_L = \overline{\boldsymbol{W}}_o \overline{\boldsymbol{Z}}_L$ を目指します．
$\widehat{\boldsymbol{H}}_L = \overline{\boldsymbol{H}}_L$ を使い，
$(\boldsymbol{W}_o+\Delta \boldsymbol{W}_o) ((\boldsymbol{W}_{2L}+\Delta \boldsymbol{W}_{2L}) \overline{\boldsymbol{H}}_L + \widehat{\boldsymbol{b}}_{2L} \mathbf{1}_N^{\top}) = \overline{\boldsymbol{W}}_o (\overline{\boldsymbol{W}}_{2L} \overline{\boldsymbol{H}}_L + \overline{\boldsymbol{b}}_{2L} \mathbf{1}_N^{\top})$
となります．
ここで，$\widehat{\boldsymbol{b}}_{2L} = (\boldsymbol{W}_o+\Delta \boldsymbol{W}_o)^{-1} \overline{\boldsymbol{W}}_o \overline{\boldsymbol{b}}_{2L}$ と設定し（$(\boldsymbol{W}_o+\Delta \boldsymbol{W}_o)$ の正則性は仮定4と補題7から保証される），
$(\boldsymbol{W}_o+\Delta \boldsymbol{W}_o) (\boldsymbol{W}_{2L}+\Delta \boldsymbol{W}_{2L}) = \overline{\boldsymbol{W}}_o \overline{\boldsymbol{W}}_{2L}$
を達成するように $\Delta \boldsymbol{W}_o$ と $\Delta \boldsymbol{W}_{2L}$ を構成します．
誤差行列は $\boldsymbol{E}_{out} = \overline{\boldsymbol{W}}_o \overline{\boldsymbol{W}}_{2L} - \boldsymbol{W}_o \boldsymbol{W}_{2L}$．
ランク $R$ が $R \geq \lceil \operatorname{rank}(\boldsymbol{E}_{out}) / 2 \rceil$ であれば，補題7により達成可能です．
定理7の $G_{L+1} = \operatorname{rank}(\overline{\boldsymbol{W}}_{o} \overline{\boldsymbol{W}}_{2L}-\boldsymbol{W}_{o} \boldsymbol{W}_{2L})$ と条件 $R \ge \lceil G_{L+1}/2 \rceil$ がこれを保証します．


## この研究における知見
* 大きなモデル（例: GPT-3よりGPT-4）ほど，LoRAでちょっと調整するだけで目標性能に届きやすくなります．
* 元のモデルがすでに目標のモデルと似ていれば，LoRAでの調整は少しで済みます．
* 元のモデルがランダムの場合，最終層の調整より，LoRAでモデル全体に少しずつ調整するほうがうまくいきます．
* LoRAで重みを調整するだけでなくバイアスも一緒に調整するとうまくいく
* Transformer系では，self-attentionの重みを調整するだけで結構よい性能がでます．
* LoRAの最適化アルゴリズムは最適ではない可能性があります．


